# Groupby, Pivot, Merge

In [295]:
import pandas as pd 
import numpy as np 

univ = ['AAPL','MSFT','BAC','GS']
dates = pd.date_range('20110101','20201231')
df = pd.DataFrame(np.random.randn(len(dates),len(univ)),index=dates,columns=univ)
df

,AAPL,MSFT,BAC,GS
2011-01-01,1.855764,-1.021861,0.442751,1.542125
2011-01-02,0.161157,-0.473939,1.549355,0.140818
2011-01-03,0.649764,-0.073461,-1.098348,-2.150049
2011-01-04,-0.692401,-0.384885,-1.248792,-1.589464
2011-01-05,0.507031,0.002092,-0.973171,0.250940
...,...,...,...,...
2020-12-27,-1.198077,-1.056570,1.895510,0.064644
2020-12-28,0.406505,0.885121,-0.174755,-0.840329
2020-12-29,0.422017,0.207292,-0.330372,-2.222286
2020-12-30,2.462412,-1.099740,-0.463891,-0.881274


In [296]:
sector = {'AAPL':'Tech','MSFT':'Tech','BAC':'Fin','GS':'Fin'}
data=[]
for dt in dates:
    for x in univ:
        data.append([x,sector[x],dt,np.random.randn(),np.random.randn()])
data = pd.DataFrame(data,columns=['ticker','sector','date','signal1','signal2'])
data

,ticker,sector,date,signal1,signal2
0,AAPL,Tech,2011-01-01,-0.459138,0.397977
1,MSFT,Tech,2011-01-01,0.460213,-0.459214
2,BAC,Fin,2011-01-01,-1.523721,-0.981242
3,GS,Fin,2011-01-01,-1.753475,0.766425
4,AAPL,Tech,2011-01-02,0.367731,1.100471
...,...,...,...,...,...
14607,GS,Fin,2020-12-30,1.757621,-1.123104
14608,AAPL,Tech,2020-12-31,0.143478,-0.037716
14609,MSFT,Tech,2020-12-31,0.776285,-0.418447
14610,BAC,Fin,2020-12-31,-0.010247,0.887626


## Groupby

In [297]:
data.groupby('ticker').dtypes

/var/folders/1k/6tgzt4fj5lz2lfymwzmv_zb00000gn/T/ipykernel_72769/19646010.py:1: FutureWarning: DataFrameGroupBy.dtypes is deprecated and will be removed in a future version. Check the dtypes on the base object instead
  data.groupby('ticker').dtypes


,ticker,sector,date,signal1,signal2
ticker,,,,,
AAPL,object,object,datetime64[ns],float64,float64
BAC,object,object,datetime64[ns],float64,float64
GS,object,object,datetime64[ns],float64,float64
MSFT,object,object,datetime64[ns],float64,float64


In [298]:
# whats the average signal of each sector on each day?
data.groupby(['sector','date'])['signal1'].mean()

sector  date      
Fin     2011-01-01   -1.638598
        2011-01-02    0.918011
        2011-01-03   -0.848468
        2011-01-04   -1.139783
        2011-01-05   -0.769352
                        ...   
Tech    2020-12-27   -0.514994
        2020-12-28    0.347375
        2020-12-29   -0.078516
        2020-12-30    0.051048
        2020-12-31    0.459882
Name: signal1, Length: 7306, dtype: float64

In [299]:
# can index before applying the function
data.groupby(['sector','date'])[['signal1']].mean()

signal1
sector date                
Fin    2011-01-01 -1.638598
       2011-01-02  0.918011
       2011-01-03 -0.848468
       2011-01-04 -1.139783
       2011-01-05 -0.769352
...                     ...
Tech   2020-12-27 -0.514994
       2020-12-28  0.347375
       2020-12-29 -0.078516
       2020-12-30  0.051048
       2020-12-31  0.459882

[7306 rows x 1 columns]

In [300]:
# use apply with groupby to pass an arbitrary function
def max_minus_min(x):
    return x.max()-x.min()

data.groupby(['sector','date'])[['signal1','signal2']].apply(max_minus_min)

signal1   signal2
sector date                          
Fin    2011-01-01  0.229755  1.747667
       2011-01-02  1.016352  1.886746
       2011-01-03  2.058465  0.995228
       2011-01-04  0.415611  1.026632
       2011-01-05  0.089806  1.403522
...                     ...       ...
Tech   2020-12-27  1.431610  0.082724
       2020-12-28  0.650820  0.864802
       2020-12-29  0.351356  0.585517
       2020-12-30  0.190615  0.925493
       2020-12-31  0.632807  0.380731

[7306 rows x 2 columns]

In [301]:
# return a dataframe instead of a series 
def demean(x):
    return x - x.mean()

data.groupby(['sector','date'])[['signal1','signal2']].apply(demean)

signal1   signal2
sector date                                
Fin    2011-01-01 2      0.114877 -0.873834
                  3     -0.114877  0.873834
       2011-01-02 6     -0.508176 -0.943373
                  7      0.508176  0.943373
       2011-01-03 10     1.029233 -0.497614
...                           ...       ...
Tech   2020-12-29 14601 -0.175678 -0.292758
       2020-12-30 14604 -0.095307  0.462747
                  14605  0.095307 -0.462747
       2020-12-31 14608 -0.316404  0.190366
                  14609  0.316404 -0.190366

[14612 rows x 2 columns]

In [302]:
df = data.groupby(['sector','date'])[['signal1','signal2']].mean()
df

signal1   signal2
sector date                          
Fin    2011-01-01 -1.638598 -0.107409
       2011-01-02  0.918011  0.153446
       2011-01-03 -0.848468 -0.074968
       2011-01-04 -1.139783  0.625450
       2011-01-05 -0.769352 -0.237389
...                     ...       ...
Tech   2020-12-27 -0.514994 -0.336006
       2020-12-28  0.347375 -0.649300
       2020-12-29 -0.078516  0.447475
       2020-12-30  0.051048  0.954582
       2020-12-31  0.459882 -0.228081

[7306 rows x 2 columns]

In [303]:
df.groupby(level=0).mean()

,signal1,signal2
sector,,
Fin,0.013408,0.029714
Tech,-0.011649,-0.021039


In [304]:
month = [x.month for x in data['date']]
data.groupby(month)[['signal1', 'signal2']].mean()

,signal1,signal2
1,0.007726,0.036868
2,-0.037588,0.047369
3,0.012966,-0.001775
4,0.043307,0.012498
5,0.027290,0.028687
6,-0.007470,-0.002304
7,-0.019820,-0.020517
8,-0.035769,0.013765
9,-0.001798,-0.031303
10,0.030629,-0.003962


In [305]:
for key,val in data.groupby('sector'):
    print (key)

Fin
Tech


In [306]:
# iterating in a groupby
for key,val in data.groupby('sector'):
    print (val)

      ticker sector       date   signal1   signal2
2        BAC    Fin 2011-01-01 -1.523721 -0.981242
3         GS    Fin 2011-01-01 -1.753475  0.766425
6        BAC    Fin 2011-01-02  0.409835 -0.789927
7         GS    Fin 2011-01-02  1.426187  1.096818
10       BAC    Fin 2011-01-03  0.180765 -0.572582
...      ...    ...        ...       ...       ...
14603     GS    Fin 2020-12-29 -1.311584 -1.533727
14606    BAC    Fin 2020-12-30 -0.262511  0.292231
14607     GS    Fin 2020-12-30  1.757621 -1.123104
14610    BAC    Fin 2020-12-31 -0.010247  0.887626
14611     GS    Fin 2020-12-31 -1.338053 -1.211901

[7306 rows x 5 columns]
      ticker sector       date   signal1   signal2
0       AAPL   Tech 2011-01-01 -0.459138  0.397977
1       MSFT   Tech 2011-01-01  0.460213 -0.459214
4       AAPL   Tech 2011-01-02  0.367731  1.100471
5       MSFT   Tech 2011-01-02 -0.015488 -0.426040
8       AAPL   Tech 2011-01-03  0.343162  0.072392
...      ...    ...        ...       ...       ...
14601 

In [307]:
data

,ticker,sector,date,signal1,signal2
0,AAPL,Tech,2011-01-01,-0.459138,0.397977
1,MSFT,Tech,2011-01-01,0.460213,-0.459214
2,BAC,Fin,2011-01-01,-1.523721,-0.981242
3,GS,Fin,2011-01-01,-1.753475,0.766425
4,AAPL,Tech,2011-01-02,0.367731,1.100471
...,...,...,...,...,...
14607,GS,Fin,2020-12-30,1.757621,-1.123104
14608,AAPL,Tech,2020-12-31,0.143478,-0.037716
14609,MSFT,Tech,2020-12-31,0.776285,-0.418447
14610,BAC,Fin,2020-12-31,-0.010247,0.887626


In [308]:
data.groupby('date')
data

,ticker,sector,date,signal1,signal2
0,AAPL,Tech,2011-01-01,-0.459138,0.397977
1,MSFT,Tech,2011-01-01,0.460213,-0.459214
2,BAC,Fin,2011-01-01,-1.523721,-0.981242
3,GS,Fin,2011-01-01,-1.753475,0.766425
4,AAPL,Tech,2011-01-02,0.367731,1.100471
...,...,...,...,...,...
14607,GS,Fin,2020-12-30,1.757621,-1.123104
14608,AAPL,Tech,2020-12-31,0.143478,-0.037716
14609,MSFT,Tech,2020-12-31,0.776285,-0.418447
14610,BAC,Fin,2020-12-31,-0.010247,0.887626


## Pivot

In [309]:
data

,ticker,sector,date,signal1,signal2
0,AAPL,Tech,2011-01-01,-0.459138,0.397977
1,MSFT,Tech,2011-01-01,0.460213,-0.459214
2,BAC,Fin,2011-01-01,-1.523721,-0.981242
3,GS,Fin,2011-01-01,-1.753475,0.766425
4,AAPL,Tech,2011-01-02,0.367731,1.100471
...,...,...,...,...,...
14607,GS,Fin,2020-12-30,1.757621,-1.123104
14608,AAPL,Tech,2020-12-31,0.143478,-0.037716
14609,MSFT,Tech,2020-12-31,0.776285,-0.418447
14610,BAC,Fin,2020-12-31,-0.010247,0.887626


In [310]:
# put Signal1 into a df with columns tickers and rows dates
df1 = data.set_index(['date','ticker'])['signal1'].unstack(level=1)
df1

ticker,AAPL,BAC,GS,MSFT
date,,,,
2011-01-01,-0.459138,-1.523721,-1.753475,0.460213
2011-01-02,0.367731,0.409835,1.426187,-0.015488
2011-01-03,0.343162,0.180765,-1.877701,0.707804
2011-01-04,0.618568,-0.931977,-1.347588,0.370359
2011-01-05,1.505484,-0.814255,-0.724449,0.341481
...,...,...,...,...
2020-12-27,0.200811,-0.213166,-0.535480,-1.230799
2020-12-28,0.672785,1.262573,-0.233524,0.021965
2020-12-29,0.097162,-0.550907,-1.311584,-0.254194


In [311]:
df2 = data.pivot_table(index='date',columns='ticker',values='signal1')
df2

ticker,AAPL,BAC,GS,MSFT
date,,,,
2011-01-01,-0.459138,-1.523721,-1.753475,0.460213
2011-01-02,0.367731,0.409835,1.426187,-0.015488
2011-01-03,0.343162,0.180765,-1.877701,0.707804
2011-01-04,0.618568,-0.931977,-1.347588,0.370359
2011-01-05,1.505484,-0.814255,-0.724449,0.341481
...,...,...,...,...
2020-12-27,0.200811,-0.213166,-0.535480,-1.230799
2020-12-28,0.672785,1.262573,-0.233524,0.021965
2020-12-29,0.097162,-0.550907,-1.311584,-0.254194


In [312]:
df1.equals(df2)

True

In [313]:
df3 = data.pivot_table(index='date',columns='sector',values='signal1')
df3

sector,Fin,Tech
date,,
2011-01-01,-1.638598,0.000537
2011-01-02,0.918011,0.176122
2011-01-03,-0.848468,0.525483
2011-01-04,-1.139783,0.494463
2011-01-05,-0.769352,0.923482
...,...,...
2020-12-27,-0.374323,-0.514994
2020-12-28,0.514525,0.347375
2020-12-29,-0.931246,-0.078516


In [314]:
df4 = data.pivot_table(index='date',columns='sector',values='signal1',aggfunc=np.mean)
df4

/var/folders/1k/6tgzt4fj5lz2lfymwzmv_zb00000gn/T/ipykernel_72769/1044090258.py:1: FutureWarning: The provided callable <function mean at 0x103d32840> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df4 = data.pivot_table(index='date',columns='sector',values='signal1',aggfunc=np.mean)


sector,Fin,Tech
date,,
2011-01-01,-1.638598,0.000537
2011-01-02,0.918011,0.176122
2011-01-03,-0.848468,0.525483
2011-01-04,-1.139783,0.494463
2011-01-05,-0.769352,0.923482
...,...,...
2020-12-27,-0.374323,-0.514994
2020-12-28,0.514525,0.347375
2020-12-29,-0.931246,-0.078516


In [315]:
df3.equals(df4)

True

In [273]:
data.pivot_table(index='date',columns='sector',values='signal1',aggfunc=np.max)


/var/folders/1k/6tgzt4fj5lz2lfymwzmv_zb00000gn/T/ipykernel_72769/3152544410.py:1: FutureWarning: The provided callable <function max at 0x103d31e40> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  data.pivot_table(index='date',columns='sector',values='signal1',aggfunc=np.max)


sector,Fin,Tech
date,,
2011-01-01,-0.185336,-0.064529
2011-01-02,2.705862,1.069650
2011-01-03,-0.907341,-0.266293
2011-01-04,0.752162,1.774520
2011-01-05,0.613212,0.767035
...,...,...
2020-12-27,-1.154839,2.374197
2020-12-28,-0.624377,-0.115745
2020-12-29,1.842279,1.218465


## Merge
Check out documentation to learn more: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

In [234]:
data2=[]
for dt in dates:
    for x in ['Tech','Fin']:
        data2.append([x,dt,np.random.randn(),np.random.randn()])
        
data2 = pd.DataFrame(data2,columns=['sector','date','signal3','signal4'])
data2

,sector,date,signal3,signal4
0,Tech,2011-01-01,1.117213,0.515167
1,Fin,2011-01-01,-1.183707,-0.197671
2,Tech,2011-01-02,-1.250880,-0.110572
3,Fin,2011-01-02,0.237281,1.236118
4,Tech,2011-01-03,-0.134949,1.893411
...,...,...,...,...
7301,Fin,2020-12-29,-0.566910,0.299913
7302,Tech,2020-12-30,0.507229,-0.470414
7303,Fin,2020-12-30,1.004549,0.308280
7304,Tech,2020-12-31,0.612075,0.836862


In [235]:
data.merge(data2,left_on=['sector','date'],right_on=['sector','date'])

,ticker,sector,date,signal1,signal2,signal3,signal4
0,AAPL,Tech,2011-01-01,-0.407612,0.391251,1.117213,0.515167
1,MSFT,Tech,2011-01-01,-0.259697,-0.185009,1.117213,0.515167
2,BAC,Fin,2011-01-01,-1.539147,0.075321,-1.183707,-0.197671
3,GS,Fin,2011-01-01,1.321674,0.300713,-1.183707,-0.197671
4,AAPL,Tech,2011-01-02,-1.605609,1.534323,-1.250880,-0.110572
...,...,...,...,...,...,...,...
14607,GS,Fin,2020-12-30,-1.091003,-1.330363,1.004549,0.308280
14608,AAPL,Tech,2020-12-31,1.545684,0.579119,0.612075,0.836862
14609,MSFT,Tech,2020-12-31,0.775471,-1.190557,0.612075,0.836862
14610,BAC,Fin,2020-12-31,0.702307,1.004321,0.509740,0.612715
